In [ ]:

from video_block import * 
from inflator import *
from triplet_loss import *
from supcon_module import *
from cus_cbs import *

In [ ]:
#export
import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from vidaug import augmentors as va

import torchvision.transforms as T


In [ ]:
# export
from modules_th.video_block import * 
from modules_th.inflator import *
from modules_th.triplet_loss import *
from modules_th.supcon_module import *
from modules_th.cus_cbs import *

# Train

In [ ]:
# export
def read_data():
    prefix = '/mnt/data/eugeniomarinelli/'
    items_path = prefix + 'UCF_experiments/training_cnn_ucf.csv'
    return pd.read_csv(items_path, index_col=0)

In [ ]:
#export 
def get_learner(df, loss='CE', l=40, size=224, n_lbl =4, n_el=2, skip=20, embs_size=128,n_views=1):
    vid_paths = df.vid_files.values

    sometimes = lambda x:va.Sometimes(0.5,x)

    vid_pip = [createVideoForm(l=l,skip=skip, form='img'),
               Resize(size, pad_mode='pad'),
               sometimes(va.HorizontalFlip()),
               va.GaussianBlur(1.),
               sometimes(va.InvertColor()),
               va.RandomRotate(10),
               lambda x: Video(x)]

    lbl_pip = [parent_label, Categorize()]           
    pip = [vid_pip, lbl_pip]
    #splits
    splits = ColSplitter('val')(df)

    # Datasets and dataloaders
    dsets = Datasets(vid_paths, pip, splits=splits)

    dls  = dsets.dataloaders(bs=n_el*n_lbl, 
                             shuffle_train=True,
                             after_item=ToTensor(),
                             after_batch=[IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)])

#     dls.valid.shuffle = True
#     dls.train.shuffle_fn = UniformizedShuffle(df.lbls.iloc[splits[0]], n_el = n_el, n_lbl= n_lbl)
#     dls.valid.shuffle_fn = UniformizedShuffle(df.lbls.iloc[splits[1]], n_el = n_el, n_lbl= n_lbl)
    
    head, body = inflate(create_head(4096, len(dls.vocab), lin_ftrs=[embs_size])), inflate(create_body(resnet50, cut=-2))


    model = nn.Sequential(body, head)

        
    learn = Learner(dls, 
                model,

                splitter=lambda model : [params(model[0]), params(model[1])], 
                loss_func=CrossEntropyLossFlat(),
               metrics=accuracy)
    
 
    
    return learn


In [ ]:
df = pd.read_csv('/mnt/data/eugeniomarinelli/UCF_experiments/training_cnn_ucf.csv')
learn = get_learner(df, size=224,l=40)

Logs_csv =   CSVLogger(fname='logs_UCF_CE_baseline.csv', append=False) 



In [ ]:

lambda x =1

SyntaxError: invalid syntax (<ipython-input-17-37e3622eb79c>, line 1)

In [ ]:
learn.fine_tune(5, cbs=[Logs_csv])
learn.export('CE_UCF_1')

epoch,train_loss,valid_loss,accuracy,time
0,2.705528,2.330142,0.292135,02:20


cbp too large (49) at 12 9
error while decoding MB 12 9
P sub_mb_type 24 out of range at 4 4
cbp too large (49) at 12 9
error while decoding MB 4 4
error while decoding MB 12 9
out of range intra chroma pred mode
error while decoding MB 6 3
out of range intra chroma pred mode
error while decoding MB 7 2
P sub_mb_type 13 out of range at 5 2
error while decoding MB 5 2
P sub_mb_type 24 out of range at 4 4
cbp too large (49) at 12 9
error while decoding MB 4 4
error while decoding MB 12 9
out of range intra chroma pred mode
error while decoding MB 6 3
out of range intra chroma pred mode
error while decoding MB 7 2
P sub_mb_type 13 out of range at 5 2
error while decoding MB 5 2
P sub_mb_type 24 out of range at 4 4
cbp too large (49) at 12 9
error while decoding MB 4 4
error while decoding MB 12 9
out of range intra chroma pred mode
error while decoding MB 6 3
out of range intra chroma pred mode
error while decoding MB 7 2
P sub_mb_type 13 out of range at 5 2
error while decoding MB 5 2
P 

KeyboardInterrupt: 

In [ ]:
learn.load('SCCE_UCF_1')
#learn.validate()

In [ ]:


learn.save('SCCE_UCF_1')

In [ ]:

?? Learner.export

# Test


In [ ]:
# Test
df = pd.read_csv('experiments_on_images/test_CALTECH101.csv')
item = df.img_files.values

test_dl = learn.dls.test_dl(item, with_labels=True) 
preds, lbls = learn.get_preds(dl= test_dl)

In [ ]:
fv = preds[1].unbind(dim=1)[0]
preds = preds[0].unbind(dim=1)[0]
print('Silhouette Score: \t', silhouette_score(fv,lbls))
print('Accuracy: \t', accuracy(preds,lbls))


In [ ]:
print(preds.shape)
print(lbls.shape)

In [ ]:
?? SumLoss

# Clustering

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import umap

In [ ]:
# Umap Embs
cmap='tab10'
reducer = umap.UMAP()

UMAPembeddings=reducer.fit_transform(fv)

# Plot

fig, ax = plt.subplots(figsize=(16,8))

scatter=ax.scatter(x=UMAPembeddings[:,0],y=UMAPembeddings[:,1], s=2, c=np.array(lbls),cmap=cmap)

plt.legend(handles=scatter.legend_elements()[0],labels=dls.vocab,
                    loc="upper right", title="elements")
plt.show()

# Test TFMS

In [ ]:
df = pd.read_csv('/mnt/data/eugeniomarinelli/UCF_experiments/train_UCF.csv')


In [ ]:
# items data

vid_paths = df.vid_files.values

sometimes = lambda x:va.Sometimes(0.5,x)

vid_pip = [createVideoForm(l=30,skip=2, form='img'),
           Resize(128, pad_mode='pad'),
           sometimes(va.HorizontalFlip()),
           va.GaussianBlur(1.), 
           sometimes(va.InvertColor()),
           lambda x: Video(x)]

lbl_pip = [parent_label, Categorize()]  
pip = [vid_pip, vid_pip, lbl_pip]

splits = RandomSplitter()(df.vid_files.values)

dsets = Datasets(vid_paths, pip, splits=splits)

In [ ]:
dls = dsets.dataloaders(bs = 8,
                  after_item = [ToTensor(), Resize(128)],
                 after_batch=[IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)],
                       shuffle_train=False)


In [ ]:
i=0
for b in dls:
    i+=1
    if not isinstance(b[0],tensor):
        print(type(b))
    print(i, end='\r')

In [ ]:

def vid_aug_transforms(size:tuple):
    size = (size,size) if isinstance(size,int) else size
    return [T.GaussianBlur(),
           RandomColorJitter(brightness=.7,contrast=.5,saturation=1),
           RandomHFlip(p=0.5)]



In [ ]:
vid = createVideoForm()('/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Testing_Normal_Videos_Anomaly/Normal_Videos_903_x264.mp4')
rsz_vid = T.Resize(128)(vid)
crop_vid = T.RandomCrop(128)(rsz_vid)

In [ ]:
rsz_vid.show()
crop_vid.show()

# Junk

In [ ]:
# Umap Reducer
reducer = umap.UMAP()
UMAPembeddings=reducer.fit_transform(fv.cpu())

In [ ]:
from sklearn.cluster import AgglomerativeClustering
seed=42
np.random.seed(seed)
cluster=AgglomerativeClustering(10)


In [ ]:
nnchain_pred=cluster.fit_predict(UMAPembeddings)
nnchain_pred.shape

In [ ]:
import matplotlib.pyplot as plt
cmap='tab10'
fig, (ax1,ax2) = plt.subplots(1,2,figsize=[20.0, 8.0])
scatter1=ax1.scatter(x=UMAPembeddings[:,0],y=UMAPembeddings[:,1], s=2, c=nnchain_pred,cmap=cmap)
scatter2=ax2.scatter(x=UMAPembeddings[:,0],y=UMAPembeddings[:,1], s=2, c=xlabels.numpy(),cmap=cmap)
legend1 = ax1.legend(*scatter1.legend_elements(),
                    loc="lower left", title="Clusters")
legend2=ax2.legend(handles=scatter2.legend_elements()[0],labels=dls.vocab,loc="lower right", title="Objects")
plt.show()

In [ ]:
df = pd.read_csv('UCF_experiments/train_UCF.csv', index_col = 0)
idxs = [lbl not in ['Testing_Normal_Videos_Anomaly' ,'Training_Normal_Videos_Anomaly' ] for lbl in df.lbls]

df = df[idxs][['vid_files', 'lbls', 'val', 'embs_path']]
splits = RandomSplitter(valid_pct=0.2)(df.vid_files.values)
df.val.loc[df.index.isin(splits[0])] = False
df.val.loc[df.index.isin(splits[1])] = True
df.head(10)

In [ ]:

    
class RandomColorJitter(Transform):
    def __init__(self,brightness=10, contrast=10, saturation=10):
          self.transform = T.ColorJitter(brightness= brightness, contrast=contrast, saturation=saturation, hue=0)
    def encodes(self, ts_vid):    
        return self.transform(ts_vid.permute(1,0,2,3)).permute(1,0,2,3)
    

In [ ]:
vid_path = 'UCF_Crimes/Videos/Abuse/Abuse001_x264.mp4'
ts_vid = CreateVideoTensor()(vid_path)
import time
start1 = time.time()
jit_vid = RandomColorJitter()(ts_vid)
print(time.time()-start1)
print(jit_vid.shape)
start2 = time.time()
jit_vid = RandomColorJitter2()(ts_vid)
print(time.time()-start2)
print(jit_vid.shape)
start3 = time.time()
jit_vid = RandomColorJitter3()(ts_vid)
print(time.time()-start3)
print(jit_vid.shape)

In [ ]:
ts_vid.select(1,2).shape

In [ ]:
import torchvision.transforms as T
img_path = 'data/funghi/porcini/00000000.jpg'
img = PILImage.create(img_path)
print(type(img))
img_ts = ToTensor()(img)
print(img_ts.shape)
tf_img=T.ColorJitter(brightness=.7,contrast=0.5,saturation=1,hue=0)(img)
tf_img


In [ ]:
?? open_image

# Building csv


In [ ]:
import pandas as pd
from fastai import *
from fastai.vision import *

items = get_files('/mnt/data/eugeniomarinelli/VIDEO/Videos', extensions='.mp4')
lbls = L(map(parent_label,vid_paths))


tr_va,test = RandomSplitter(valid_pct=0.1)(items)

train,val = RandomSplitter(valid_pct=0.15)(items[tr_va])
is_val = L(False)*len(tr_va)
is_val[val] = True
df = pd.DataFrame(list(zip(items[tr_va],lbls[tr_va],is_val)), columns=['vid_files','lbls','val'])
df_test = pd.DataFrame(list(zip(items[test],lbls[test])), columns=['vid_files','lbls'])
df.head()

In [ ]:
df.to_csv('UCF_experiments/train_UCF.csv', index=False)
df_test.to_csv('UCF_experiments/test_UCF.csv',index = False)

In [ ]:
import torchvision.transforms as T 

In [ ]:

mean = [0.43216, 0.394666, 0.37645]
std = [0.22803, 0.22145, 0.216989]
pipl = Pipeline([TensorVideo, T.Resize((224,224)),IntToFloatTensor(), Normalize.from_stats(mean,std)])
snips = L(chunked(vid, 100)).map(pipl)


In [ ]:
vid, aud, dic = read_video('Training_Normal_Videos_Anomaly/Normal_Videos200_x264.mp4',pts_unit='sec')
ten_list = L(chunked(vid, 100)).map(torch.stack)
correct_form = L(ten_list[i].permute(3,0,1,2) for i in range(len(ten_list))).
correct_form.map(pipl)[0]
                 

In [ ]:
type(snips[0])

# Show Video

In [ ]:

f = time.perf_counter()
vid, aud, dic = read_video('mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Assault/Assault010_x264.mp4',pts_unit='sec')
print(time.perf_counter()-f)

In [ ]:
import random
vid,_,_ = read_video(str(random.choice(df.vid_files)),pts_unit='sec')
vid.shape

In [ ]:
#export
#################
# SHOW VIDEO    #
#################
import time
import pylab as pl
from torchvision.io import read_video
from IPython.display import clear_output, display
def play_video(path='/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Burglary/Burglary003_x264.mp4', skip=20):
    vid, aud, dic = read_video(path,pts_unit='sec')
    for i in range(vid.size(0)):

        if i%skip==0:
            clear_output(wait=True)
            TensorImage(vid[i]).show()
            display(pl.gcf())
            time.sleep(0.01)
        print(i)
#clear_output()

In [ ]:
play_video(str(random.choice(df.vid_files)))

# Export

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_charades.ipynb.
Converted 05_ucf.ipynb.
Converted 06_distributed_inflated_NN.ipynb.
Converted 07_read_video_function.ipynb.
Converted 08_experiments_on_images.ipynb.
Converted 09_create_embeddings.ipynb.
Converted CustomCallbacks.ipynb.
Converted SupConModule.ipynb.
Converted Transformers.ipynb.
Converted UCF_CEL.ipynb.
Converted UCF_SupCon.ipynb.
Converted distributed_training.ipynb.
Converted index.ipynb.
Converted mixedloss_images.ipynb.


In [ ]:
# default_exp ucf_ce